# Entrenamiento de un modelo de traducción con OpenNMT

Vamos a hacer el preprocesamiento indicado en el tutorial de OpenNMT que se encuentra en la documentación oficial: https://github.com/ymoslem/OpenNMT-Tutorial.

Luego vamos a hacer el entrenamiento como se indica en el mismo tutorial (esta parte del tutorial está en https://github.com/ymoslem/OpenNMT-Tutorial/blob/main/2-NMT-Training.ipynb)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Vamos a estar trabajando sobre una carpeta en nuestro drive, llamada nmt-practica8
%cd /content/drive/MyDrive/nmt-practica8/

/content/drive/MyDrive/nmt-practica8


Además tenemos que haber creado una carpeta llamada *models* y otra llamada *built_vocab*, dentro de *nmt-test*.

In [4]:
!ls

built_vocab  models  nmt-practica8.ipynb


Primero tenemos que cambiar el entorno a GPU. Si no está en GPU, tenemos que cambiar el entorno de ejecución en colab, y luego reiniciarlo.

In [5]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f53c81a4-f5b8-c19c-9e10-a76c25ce3494)


In [6]:
# Check if the GPU is visable to PyTorch
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


## Descargar dataset
Nuestro corpus van a ser los archivos del corpus Nahuatl-Español proporcionado para el shared task de Machine Translation del AmericasNLP 2021 (los encontramos en https://github.com/AmericasNLP/americasnlp2021/tree/main/data/nahuatl-spanish y https://github.com/AmericasNLP/americasnlp2021/blob/main/test_data/test.nah).

In [7]:
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.es
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.nah
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/test.es
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/train.nah
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/train.es
!wget --no-cache --backups=1 https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/test_data/test.nah

--2023-11-27 02:01:41--  https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.es
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36502 (36K) [text/plain]
Failed to rename dev.es to dev.es.1: (2) No such file or directory
Saving to: ‘dev.es’

dev.es              100%[===================>]  35.65K  --.-KB/s    in 0.005s  

2023-11-27 02:01:41 (7.25 MB/s) - ‘dev.es’ saved [36502/36502]

--2023-11-27 02:01:41--  https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.nah
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connect

## Preprocesamiento

Para usar OpenNMT necesitamos usar los módulos de preprocesamiento que están en `https://github.com/ymoslem/MT-Preparation`. Y luego vamos a seguir los pasos que nos dan en su readme.

In [8]:
# Clonamos el repositorio para herramientas de preprocesamiento
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 268 (delta 133), reused 189 (delta 97), pack-reused 0
Receiving objects: 100% (268/268), 69.06 KiB | 552.00 KiB/s, done.
Resolving deltas: 100% (133/133), done.


### Instalar requerimientos
Hay que instalar:
- numpy
- pandas
- sentencepiece

(estos nombres de paquetes están en el archivo MT-Preparation/requirements.txt)

In [4]:
!pip3 install -r MT-Preparation/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00


### Filtrado de datos
Tenemos que correr el script `MT-Preparation/filtering/filter.py`, dándole como argumentos dos archivos: uno para la fuente (source), y otro como target.

En nuestro caso vamos a usar los archivos `train.nah` y `train.es`.

Esto crea los archivos `train.nah-filtered.nah` y `train.es-filtered.es`:

In [10]:
# Arguments: source file, target file, source language, target language
!python3 MT-Preparation/filtering/filter.py train.nah train.es nah es

Dataframe shape (rows, columns): (16145, 2)
--- Rows with Empty Cells Deleted	--> Rows: 16062
--- Duplicates Deleted			--> Rows: 15448
--- Source-Copied Rows Deleted		--> Rows: 15373
--- Too Long Source/Target Deleted	--> Rows: 14170
--- HTML Removed			--> Rows: 14170
--- Rows will remain in true-cased	--> Rows: 14170
--- Rows with Empty Cells Deleted	--> Rows: 14170
--- Rows Shuffled			--> Rows: 14170
--- Source Saved: train.nah-filtered.nah
--- Target Saved: train.es-filtered.es


Hacemos lo mismo con los archivos dev y test.

In [6]:
!python3 MT-Preparation/filtering/filter.py dev.nah dev.es nah es

Dataframe shape (rows, columns): (672, 2)
--- Rows with Empty Cells Deleted	--> Rows: 671
--- Duplicates Deleted			--> Rows: 671
--- Source-Copied Rows Deleted		--> Rows: 671
--- Too Long Source/Target Deleted	--> Rows: 613
--- HTML Removed			--> Rows: 613
--- Rows will remain in true-cased	--> Rows: 613
--- Rows with Empty Cells Deleted	--> Rows: 613
--- Rows Shuffled			--> Rows: 613
--- Source Saved: dev.nah-filtered.nah
--- Target Saved: dev.es-filtered.es


In [7]:
!python3 MT-Preparation/filtering/filter.py test.nah test.es nah es

Dataframe shape (rows, columns): (1003, 2)
--- Rows with Empty Cells Deleted	--> Rows: 996
--- Duplicates Deleted			--> Rows: 996
--- Source-Copied Rows Deleted		--> Rows: 996
--- Too Long Source/Target Deleted	--> Rows: 879
--- HTML Removed			--> Rows: 879
--- Rows will remain in true-cased	--> Rows: 879
--- Rows with Empty Cells Deleted	--> Rows: 879
--- Rows Shuffled			--> Rows: 879
--- Source Saved: test.nah-filtered.nah
--- Target Saved: test.es-filtered.es


### Tokenización por subpalabras
#### Creando los modelos
En MT-Preparation/subwording/ hay varios scripts para entrenar modelos de tokenización por subpalabras. Usaremos el script 1-train_unigram.py.

Esto crea los archivos `source.vocab`, `target.vocab`, `source.model` y `target.model`.


In [11]:
# Train a SentencePiece model for subword tokenization
!python MT-Preparation/subwording/1-train_unigram.py train.nah-filtered.nah train.es-filtered.es

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=train.nah-filtered.nah --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --split_digits=true
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.nah-filtered.nah
  input_format: 
  model_prefix: source
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 0
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  e

#### Tokenizando los archivos (con los modelos que creamos)
Ahora usamos los modelos que creamos para tokenizar nuestros archivos.

Corremos: `python3 subword.py <sp_source_model_path> <sp_target_model_path> <source_file_path> <target_file_path>`

In [5]:
# Ahora tokenizamos nuestras oraciones en el train set usando subwords
!python MT-Preparation/subwording/2-subword.py source.model target.model train.nah-filtered.nah train.es-filtered.es

Source Model: source.model
Target Model: target.model
Source Dataset: train.nah-filtered.nah
Target Dataset: train.es-filtered.es
Done subwording the source file! Output: train.nah-filtered.nah.subword
Done subwording the target file! Output: train.es-filtered.es.subword


In [8]:
# También tokenizamos las oraciones en el dev set y en el test set
!python MT-Preparation/subwording/2-subword.py source.model target.model dev.nah-filtered.nah dev.es-filtered.es

Source Model: source.model
Target Model: target.model
Source Dataset: dev.nah-filtered.nah
Target Dataset: dev.es-filtered.es
Done subwording the source file! Output: dev.nah-filtered.nah.subword
Done subwording the target file! Output: dev.es-filtered.es.subword


In [9]:
!python MT-Preparation/subwording/2-subword.py source.model target.model test.nah-filtered.nah test.es-filtered.es

Source Model: source.model
Target Model: target.model
Source Dataset: test.nah-filtered.nah
Target Dataset: test.es-filtered.es
Done subwording the source file! Output: test.nah-filtered.nah.subword
Done subwording the target file! Output: test.es-filtered.es.subword


## Entrenando el modelo

Instalamos OpenNMT si no lo tenemos.

In [10]:
!pip install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 32.1 MB/s eta 0:00:00


Los pasos para esta parte se muestran en el tutorial de OpenNMT (https://github.com/ymoslem/OpenNMT-Tutorial/blob/main/2-NMT-Training.ipynb).

### Creando el archivo de configuración

**Importante:** antes de correr esto hay que crear una carpeta llamada `models` dentro de `nmt-practica8`.

In [11]:
# Creación del archivo de configuración
# Usando valores pequeños en vista de que tenemos un corpus limitado
# Para datasets grandes deberian aumentar los valores:
# train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint
SRC_DATA_NAME = "nah-filtered.nah.subword"
TARGET_DATA_NAME = "es-filtered.es.subword"

config = f'''# config.yaml

## Where the samples will be written
save_data: run

# Rutas de archivos de entrenamiento
#(previamente aplicado subword tokenization)
data:
    corpus_1:
        path_src: train.{SRC_DATA_NAME}
        path_tgt: train.{TARGET_DATA_NAME}
        transforms: [filtertoolong]
    valid:
        path_src: dev.{SRC_DATA_NAME}
        path_tgt: dev.{TARGET_DATA_NAME}
        transforms: [filtertoolong]

# Vocabularios (serán generados por `onmt_build_vocab`)
src_vocab: built_vocab/source.vocab
tgt_vocab: built_vocab/target.vocab

# Tamaño del vocabulario
#(debe concordar con el parametro usado en el algoritmo de subword tokenization)
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filtrado sentencias de longitud mayor a n
# actuara si [filtertoolong] está presente
src_seq_length: 150
src_seq_length: 150

# Tokenizadores
src_subword_model: source.model
tgt_subword_model: target.model

# Archivos donde se guardaran los logs y los checkpoints de modelos
log_file: train.log
save_model: models/model.enes

# Condición de paro si no se obtienen mejoras significativas
# despues de n validaciones
early_stopping: 4

# Guarda un checkpoint del modelo cada n steps
save_checkpoint_steps: 1000

# Mantiene los n ultimos checkpoints
keep_checkpoint: 3

# Reproductibilidad
seed: 3435

# Entrena el modelo maximo n steps
# Default: 100,000
train_steps: 3000

# Corre el set de validaciones (*.dev) despues de n steps
# Defatul: 10,000
valid_steps: 1000

warmup_steps: 1000
report_every: 100

# Numero de GPUs y sus ids
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0
batch_type: "tokens"
batch_size: 4096
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Configuración del optimizador
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Configuración del Modelo
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("/content/drive/MyDrive/nmt-practica8/config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

### Construimos el vocabulario

In [12]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [13]:
# Build Vocabulary:
# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster
%%time
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

2023-11-27 05:29:32.437078: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 05:29:32.437138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 05:29:32.437192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 05:29:32.448343: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 05:29:33.889520: W tensorflow/compiler/

### Entrenamiento

Ahora sí, entrenamos:

In [14]:
%%time
!onmt_train -config config.yaml

2023-11-27 05:30:02.965172: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 05:30:02.965230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 05:30:02.965267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 05:30:02.973625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 05:30:04.097061: W tensorflow/compiler/

## Traducción

In [15]:
%%time
!onmt_translate -model models/model.enes_step_3000.pt -src test.nah-filtered.nah.subword -output test-practice.translated -gpu 0 -min_length 1

2023-11-27 07:15:07.883897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 07:15:07.883953: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 07:15:07.883984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 07:15:07.891915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 07:15:09.891170: W tensorflow/compiler/

In [16]:
# Mostramos algunas oraciones antes de su traducción
!tail -n 40 test.es-filtered.es.subword

▁ ¿ T e ▁gusta ron ▁u ▁o dias te ▁cierto s ▁libro s ?
▁Me ▁dijo ▁qué ▁necesita ▁con ▁exact itud ▁y ▁que ▁lo ▁necesitaba ▁para ▁hoy .
▁ É l ▁es ▁definitiva mente ▁lo ▁suficiente mente ▁fuerte ▁como ▁para ▁que ▁yo ▁lo ▁escuche .
▁Toda s ▁las ▁puerta s ▁estaba n ▁abierta s .
▁Teng o ▁una ▁hija , ▁una ▁niña ▁de ▁dieci ocho ▁mes es .
▁La ▁Ba se ▁A érea ▁La ug h li n ▁cerr ó ▁por ▁un ▁tiempo .
▁In c l uso ▁si ▁la ▁a eron ave ▁estaba ▁en ▁llama s , ▁ ¿ por ▁qué ▁se ▁que maría ▁y ▁se ▁de rre ti ría ▁si ▁tiene ▁un ▁compone nte ▁de ▁pl omo ▁para ▁que ▁la ▁ radi ación ▁se ▁fi l tre ?
▁Este ▁parece ▁más ▁si mpl ifica do , ▁como ▁el ▁que ▁ella ▁me ▁había ▁dado ▁estaba ▁todo ▁de talla do ▁y ▁complicad o , ▁y ▁el ▁segundo ▁era ▁todo ▁simple .
▁Hay ▁mucha s ▁historia s ▁que ▁se ▁cuenta n .
▁Bueno , ▁era ▁casi ▁lo ▁bastante ▁viejo ▁como ▁para ▁ser ▁su ▁papá .
▁Fue ▁molesto ▁cuando ▁cubri ó ▁mis ▁zapat os .
▁Está ▁habla ndo ▁por ▁un ▁ i P hon e .
▁Nació ▁en ▁mil ▁ochocient os ▁ochenta ▁y ▁algo , ▁pud o 

In [23]:
# Para mejor lectura, las reconstruimos a partir de las subapalabras
!python MT-Preparation/subwording/3-desubword.py target.model test.es-filtered.es.subword

Done desubwording! Output: test.es-filtered.es.subword.desubword


In [18]:
# Ahora después de ser traducidas
!tail -n 40 test-practice.translated

▁ ¿ ▁Por ▁qué ▁se ▁van ▁a ▁aborrecer ▁?
▁T ambién ▁sirve ▁contra ▁el ▁empacho ▁.
▁...
▁Todo s ▁están ▁tirado s ▁de lante ▁de ▁ellos ▁.
▁... ▁en ▁el ▁octavo ▁mes ▁de ▁agost o ▁.
▁Don de ▁estaba n ▁muchos ▁niño s ▁cuando ▁se ▁dieron ▁muerte ▁,
▁Si ▁hubiera n ▁ido ▁a ▁hacer ▁la ▁leña ▁. ▁ ¿ ▁Qué ▁piensa ▁nada ▁? ▁, ▁ ¿ ▁o ▁si ▁ acaso ▁no ▁es ▁verdad ▁? ▁A quí ▁está ▁su ▁casa ▁, ▁por ▁lo ▁que ▁se ▁bebe ▁, ▁se ▁da ▁a trá s ▁de ▁la ▁tierra ▁? ▁, ▁se ▁lava ▁el ▁templo ▁.
▁Puede ▁ser ▁que ▁no ▁es ▁anciano ▁, ▁sin o ▁que ▁a ▁una ▁anciana ▁.
▁Hay ▁de ▁la ▁gente ▁en ▁aquel ▁tiempo ▁, ▁es ▁bueno ▁.
▁Bi en ▁limpia s ▁y ▁sin o ▁que ▁se ▁haga n ▁el ▁corazón ▁.
▁Ya ▁no ▁hay ▁que ▁trabajar
▁Por ▁esta ▁ razón ▁discut ieron ▁y ▁se ▁alterar on ▁las ▁flores
▁... ▁se ▁ 1 9 ▁ Á catl ▁ 1 9 8 1 ▁.
▁ 1 ⁄ 2 ▁kilo ▁de ▁verdolagas
▁S ólo ▁eso ▁es ▁que ▁cuando ▁no ▁haya ▁bueno ▁el ▁trabajo
▁Qué ▁impone nte ▁es ▁el ▁aspecto ▁de ▁la ▁gran ▁ciudad ▁de ▁México Tenochtitlan ▁.
▁Lo ▁he ▁escondido ▁a ▁esta ▁vida
▁Y ▁si ▁n

In [19]:
# Para mejor lectura, las reconstruimos a partir de las subapalabras
!python MT-Preparation/subwording/3-desubword.py target.model test-practice.translated

Done desubwording! Output: test-practice.translated.desubword


In [20]:
# Oraciones traducidas con las palabras reconstruidas
!tail -n 40 test-practice.translated.desubword

¿ Por qué se van a aborrecer ?
También sirve contra el empacho .
...
Todos están tirados delante de ellos .
... en el octavo mes de agosto .
Donde estaban muchos niños cuando se dieron muerte ,
Si hubieran ido a hacer la leña . ¿ Qué piensa nada ? , ¿ o si acaso no es verdad ? Aquí está su casa , por lo que se bebe , se da atrás de la tierra ? , se lava el templo .
Puede ser que no es anciano , sino que a una anciana .
Hay de la gente en aquel tiempo , es bueno .
Bien limpias y sino que se hagan el corazón .
Ya no hay que trabajar
Por esta razón discutieron y se alteraron las flores
... se 19 Ácatl 1981 .
1⁄2 kilo de verdolagas
Sólo eso es que cuando no haya bueno el trabajo
Qué imponente es el aspecto de la gran ciudad de MéxicoTenochtitlan .
Lo he escondido a esta vida
Y si no me pongo de pie , sino que otra persona se ha de pagar el corazón , 20 centavos para llevarlos a la familia del cabildo seglar , por la noche .
solo en la tierra no vaya a regresar a la tierra .
Debe uno estar 

## Evaluación del modelo



In [21]:
!pip install sacrebleu

In [24]:
with open('test.es-filtered.es.subword.desubword') as f:
    test_desubworded = f.readlines()

print(len(test_desubworded))

879


In [25]:
test_desubworded[:5]

['Solo te dejan volar U2 el primer día.\n',
 'oh, eso de verdad no es realmente, oh consistente\n',
 'Ella tenía una piel increíblemente oscura.\n',
 'Ella dijo que sabía más sobre la vida.\n',
 'Realmente estoy haciendo lo mejor que puedo.\n']

In [26]:
with open('test-practice.translated.desubword') as f:
    predicted_test_desubworded = f.readlines()

print(len(predicted_test_desubworded))
predicted_test_desubworded[:5]

879


['“ Ahora te voy a echar dentro de unos cuantos20 , voy a hacer la guerra ” .\n',
 'Cierto\n',
 'Así trabajo se pegó su piel de la cabeza y así .\n',
 'De la misma manera que sólo [ la mujer ]\n',
 'Dónde me lo hago en esta tierra ,\n']

In [31]:
import sacrebleu

# Ahora obtenemos la BLUE score y el chrF usando el mismo método que se usó en la shared task (en el archivo evaluate.py):
print("BLEU:", sacrebleu.corpus_bleu(predicted_test_desubworded, test_desubworded))
print("ChrF:", sacrebleu.corpus_chrf(predicted_test_desubworded, test_desubworded))

BLEU: BLEU = 0.77 13.1/0.5/0.3/0.2 (BP = 1.000 ratio = 8.917 hyp_len = 107 ref_len = 12)
ChrF: chrF2 = 13.92
